In [1]:
import pandas as pd
import numpy as np
import catboost as cb

In [2]:
url = 'https://github.com/mattharrison/datasets/raw/master/data/2020-jetbrains-python-survey.csv'

In [3]:
jb = pd.read_csv(url)

/tmp/ipykernel_47/1530167600.py:1: DtypeWarning: Columns (152) have mixed types. Specify dtype option on import or set low_memory=False.
  jb = pd.read_csv(url)


In [4]:
jb

,is.python.main,other.lang.None,other.lang.Java,other.lang.JavaScript,other.lang.C/C++,other.lang.PHP,other.lang.C#,other.lang.Ruby,other.lang.Bash / Shell,other.lang.Objective-C,...,job.role.Technical support,job.role.Data analyst,job.role.Business analyst,job.role.Team lead,job.role.Product manager,job.role.CIO / CEO / CTO,job.role.Systems analyst,job.role.Other,age,country.live
0,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bash / Shell,NaN,...,NaN,NaN,Business analyst,NaN,NaN,NaN,NaN,NaN,30–39,NaN
1,Yes,NaN,Java,JavaScript,NaN,NaN,C#,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21–29,India
2,Yes,NaN,NaN,NaN,C/C++,NaN,NaN,NaN,Bash / Shell,NaN,...,Technical support,Data analyst,NaN,Team lead,NaN,NaN,NaN,NaN,30–39,United States
3,Yes,NaN,NaN,JavaScript,NaN,NaN,NaN,NaN,Bash / Shell,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Yes,NaN,Java,JavaScript,C/C++,NaN,NaN,NaN,Bash / Shell,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21–29,Italy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54457,Yes,NaN,NaN,NaN,C/C++,NaN,NaN,NaN,Bash / Shell,Objective-C,...,NaN,NaN,NaN,NaN,NaN,NaN,Systems analyst,NaN,21–29,Russian Federation
54458,Yes,NaN,NaN,JavaScript,NaN,NaN,NaN,NaN,Bash / Shell,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54459,Yes,NaN,NaN,JavaScript,NaN,PHP,NaN,NaN,Bash / Shell,NaN,...,NaN,NaN,NaN,NaN,NaN,CIO / CEO / CTO,NaN,NaN,21–29,Russian Federation
54460,Yes,NaN,NaN,JavaScript,C/C++,PHP,NaN,NaN,Bash / Shell,NaN,...,NaN,Data analyst,NaN,NaN,NaN,NaN,NaN,NaN,30–39,Spain


In [5]:
import collections

In [6]:
counter = collections.defaultdict(list)

In [7]:
for col in sorted(jb.columns):
    period_count = col.count('.')
    if period_count >= 2:
        part_end = 2
    else:
        part_end = 1
    parts = col.split('.')[:part_end]
    counter['.'.join(parts)].append(col)

In [8]:
uniq_cols = []

In [9]:
for cols in counter.values():
    if len(cols) == 1:
        uniq_cols.extend(cols)

In [10]:
uniq_cols

['age',
 'are.you.datascientist',
 'company.size',
 'country.live',
 'employment.status',
 'first.learn.about.main.ide',
 'how.often.use.main.ide',
 'ide.main',
 'is.python.main',
 'job.team',
 'main.purposes',
 'missing.features.main.ide',
 'nps.main.ide',
 'python.years',
 'python2.version.most',
 'python3.version.most',
 'several.projects',
 'team.size',
 'use.python.most',
 'years.of.coding']

In [11]:
(jb
[uniq_cols]
    .rename(columns=lambda c: c.replace('.', '_'))
    .age
    .value_counts(dropna=False)
)

age
NaN            29701
21–29           9710
30–39           7512
40–49           3010
18–20           2567
50–59           1374
60 or older      588
Name: count, dtype: int64

In [12]:
(jb
[uniq_cols]
    .rename(columns=lambda c: c.replace('.', '_'))
    .age
    .str.slice(0, 2)
    .astype(float)
)

0        30.0
1        21.0
2        30.0
3         NaN
4        21.0
         ... 
54457    21.0
54458     NaN
54459    21.0
54460    30.0
54461    21.0
Name: age, Length: 54462, dtype: float64

In [13]:
(jb
[uniq_cols]
    .rename(columns=lambda c: c.replace('.', '_'))
    .age
    .str.slice(0,2)
    .astype(float)
    .astype('Int64')
)

0          30
1          21
2          30
3        <NA>
4          21
         ... 
54457      21
54458    <NA>
54459      21
54460      30
54461      21
Name: age, Length: 54462, dtype: Int64

In [14]:
(jb
[uniq_cols]
    .rename(columns=lambda c: c.replace('.', '_'))
    .assign(age=lambda df_:df_.age
           .str.slice(0,2)
           .astype(float)
           .astype('Int64'))
)

,age,are_you_datascientist,company_size,country_live,employment_status,first_learn_about_main_ide,how_often_use_main_ide,ide_main,is_python_main,job_team,main_purposes,missing_features_main_ide,nps_main_ide,python_years,python2_version_most,python3_version_most,several_projects,team_size,use_python_most,years_of_coding
0,30,NaN,Just me,NaN,Partially employed by a company / organization,Conference / User Group,Weekly,PyCharm Community Edition,Yes,Work as an external consultant or trainer,For work,"No, it has all the features I need",3.0,3–5 years,NaN,Python 3_7,"Yes, I work on many different projects",NaN,NaN,1–2 years
1,21,Yes,"More than 5,000",India,Fully employed by a company / organization,School / University,Daily,VS Code,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",8.0,3–5 years,NaN,Python 3_6,"Yes, I work on one main and several side projects",2-7 people,Software prototyping,3–5 years
2,30,No,"More than 5,000",United States,Fully employed by a company / organization,Friend / Colleague,Daily,Vim,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",10.0,3–5 years,NaN,Python 3_6,"Yes, I work on one main and several side projects",NaN,DevOps / System administration / Writing autom...,3–5 years
3,<NA>,NaN,NaN,NaN,NaN,Friend / Colleague,Daily,PyCharm Professional Edition,Yes,NaN,Both for work and personal,Yes – Please list:,10.0,11+ years,NaN,Python 3_8,"Yes, I work on many different projects",NaN,Web development,11+ years
4,21,NaN,NaN,Italy,Student,Search engines,Daily,VS Code,Yes,Work on your own project(s) independently,"For personal, educational or side projects","No, it has all the features I need",10.0,1–2 years,NaN,Python 3_8,"Yes, I work on one main and several side projects",NaN,Web development,Less than 1 year
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54457,21,No,2–10,Russian Federation,Fully employed by a company / organization,School / University,Daily,Vim,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",10.0,6–10 years,NaN,Python 3_6,"Yes, I work on many different projects",NaN,Data analysis,1–2 years
54458,<NA>,No,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Both for work and personal,NaN,NaN,3–5 years,NaN,Python 3_7,NaN,NaN,Web development,1–2 years
54459,21,NaN,Just me,Russian Federation,Self-employed (a person earning income directl...,Friend / Colleague,Daily,PyCharm Professional Edition,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",10.0,3–5 years,NaN,Python 3_7,"Yes, I work on many different projects",2-7 people,Web development,6–10 years
54460,30,Yes,51–500,Spain,Fully employed by a company / organization,Search engines,Daily,Other,Yes,Work on your own project(s) independently,Both for work and personal,Yes – Please list:,3.0,6–10 years,NaN,Python 3_7,"Yes, I work on many different projects",NaN,Data analysis,3–5 years


In [16]:
(jb
[uniq_cols]
    .rename(columns=lambda c: c.replace('.', '_'))
    .assign(age=lambda df_:df_.age.str.slice(0, 2)
           .astype(float).astype('Int64'),
           are_you_datascientist=lambda df_:df_.are_you_datascientist
           .replace({'Yes': True, 'No': False, np.nan: False})
           )
    .are_you_datascientist
)

0        False
1         True
2        False
3        False
4        False
         ...  
54457    False
54458    False
54459    False
54460     True
54461    False
Name: are_you_datascientist, Length: 54462, dtype: object

In [17]:
(jb
[uniq_cols]
    .rename(columns=lambda c: c.replace('.', '_'))
    .assign(age=lambda df_:df_.age.str.slice(0,2)
           .astype(float).astype('Int64'),
           are_you_datascientist=lambda df_:df_.are_you_datascientist
           .replace({'Yes': True, 'No': False, np.nan: False})
           )
    .company_size
    .value_counts(dropna=False)
)

company_size
NaN                35037
51–500              4608
More than 5,000     3635
11–50               3507
2–10                2558
1,001–5,000         1934
Just me             1492
501–1,000           1165
Not sure             526
Name: count, dtype: int64

In [18]:
jb2 = (jb
 [uniq_cols]
 .rename(columns=lambda c: c.replace('.', '_'))
 .assign(age=lambda df_:df_.age.str.slice(0,2).astype(float)
             .astype('Int64'),
         are_you_datascientist=lambda df_:df_.are_you_datascientist
             .replace({'Yes': True, 'No': False, np.nan: False}),
         company_size=lambda df_:df_.company_size.replace({
             'Just me': 1, 'Not sure': np.nan, 
             'More than 5,000': 5000, '2–10': 2, '11–50':11,
             '51–500': 51, '501–1,000':501,
             '1,001–5,000':1001}).astype('Int64'),
         country_live=lambda df_:df_.country_live.astype('category'),
         employment_status=lambda df_:df_.employment_status
              .fillna('Other').astype('category'),
         is_python_main=lambda df_:df_.is_python_main
              .astype('category'),
         team_size=lambda df_:df_.team_size
             .str.split(r'-', n=1, expand=True)
             .iloc[:,0].replace('More than 40 people', 41)
             .where(df_.company_size!=1, 1).astype(float),
         years_of_coding=lambda df_:df_.years_of_coding
             .replace('Less than 1 year', .5).str.extract(r'(\d+)')
             .astype(float),
         python_years=lambda df_:df_.python_years
             .replace('Less than 1 year', .5).str.extract(r'(\d+)')
             .astype(float),
         python3_ver=lambda df_:df_.python3_version_most
              .str.replace('_', '.').str.extract(r'(\d\.\d)')
              .astype(float),
         use_python_most=lambda df_:df_.use_python_most
              .fillna('Unknown')
        )
    .drop(columns=['python2_version_most'])
)

In [19]:
jb2

,age,are_you_datascientist,company_size,country_live,employment_status,first_learn_about_main_ide,how_often_use_main_ide,ide_main,is_python_main,job_team,main_purposes,missing_features_main_ide,nps_main_ide,python_years,python3_version_most,several_projects,team_size,use_python_most,years_of_coding,python3_ver
0,30,False,1,NaN,Partially employed by a company / organization,Conference / User Group,Weekly,PyCharm Community Edition,Yes,Work as an external consultant or trainer,For work,"No, it has all the features I need",3.0,3.0,Python 3_7,"Yes, I work on many different projects",1.0,Unknown,1.0,3.7
1,21,True,5000,India,Fully employed by a company / organization,School / University,Daily,VS Code,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",8.0,3.0,Python 3_6,"Yes, I work on one main and several side projects",2.0,Software prototyping,3.0,3.6
2,30,False,5000,United States,Fully employed by a company / organization,Friend / Colleague,Daily,Vim,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",10.0,3.0,Python 3_6,"Yes, I work on one main and several side projects",NaN,DevOps / System administration / Writing autom...,3.0,3.6
3,<NA>,False,<NA>,NaN,Other,Friend / Colleague,Daily,PyCharm Professional Edition,Yes,NaN,Both for work and personal,Yes – Please list:,10.0,11.0,Python 3_8,"Yes, I work on many different projects",1.0,Web development,11.0,3.8
4,21,False,<NA>,Italy,Student,Search engines,Daily,VS Code,Yes,Work on your own project(s) independently,"For personal, educational or side projects","No, it has all the features I need",10.0,1.0,Python 3_8,"Yes, I work on one main and several side projects",1.0,Web development,NaN,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54457,21,False,2,Russian Federation,Fully employed by a company / organization,School / University,Daily,Vim,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",10.0,6.0,Python 3_6,"Yes, I work on many different projects",NaN,Data analysis,1.0,3.6
54458,<NA>,False,<NA>,NaN,Other,NaN,NaN,NaN,Yes,NaN,Both for work and personal,NaN,NaN,3.0,Python 3_7,NaN,1.0,Web development,1.0,3.7
54459,21,False,1,Russian Federation,Self-employed (a person earning income directl...,Friend / Colleague,Daily,PyCharm Professional Edition,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",10.0,3.0,Python 3_7,"Yes, I work on many different projects",1.0,Web development,6.0,3.7
54460,30,True,51,Spain,Fully employed by a company / organization,Search engines,Daily,Other,Yes,Work on your own project(s) independently,Both for work and personal,Yes – Please list:,3.0,6.0,Python 3_7,"Yes, I work on many different projects",NaN,Data analysis,3.0,3.7


In [20]:
jb2.columns

Index(['age', 'are_you_datascientist', 'company_size', 'country_live',
       'employment_status', 'first_learn_about_main_ide',
       'how_often_use_main_ide', 'ide_main', 'is_python_main', 'job_team',
       'main_purposes', 'missing_features_main_ide', 'nps_main_ide',
       'python_years', 'python3_version_most', 'several_projects', 'team_size',
       'use_python_most', 'years_of_coding', 'python3_ver'],
      dtype='object')

In [21]:
(jb2
.query('team_size.isna()')
.employment_status
.value_counts(dropna=False)
)

employment_status
Fully employed by a company / organization                                                        5279
Working student                                                                                    696
Partially employed by a company / organization                                                     482
Self-employed (a person earning income directly from one's own business, trade, or profession)     430
Freelancer (a person pursuing a profession without a long-term commitment to any one employer)       0
Other                                                                                                0
Retired                                                                                              0
Student                                                                                              0
Name: count, dtype: int64

In [22]:
import catboost as cb
import numpy as np

In [23]:
def prep_for_ml(df):
    # remove pandas types
    return (df
           .assign(**{col:df[col].astype(float)
                     for col in df.select_dtypes('number')},
                  **{col:df[col].astype(str).fillna('')
                    for col in df.select_dtypes(['object', 'category'])})
           )

In [26]:
def predict_col(df, col):
    df = prep_for_ml(df)
    missing = df.query(f'~{col}.isna()')
    cat_idx = [i for i, typ in enumerate(df.drop(columns=[col]).dtypes)
              if str(typ) == 'object']
    X = (missing
        .drop(columns=[col])
        .values
        )
    y = missing[col]
    model = cb.CatBoostRegressor(iterations=20, cat_features=cat_idx)
    model.fit(X,y, cat_features=cat_idx)
    pred = model.predict(df.drop(columns=[col]))
    return df[col].where(~df[col].isna(), pred)

In [27]:
jb2 = (jb
  [uniq_cols]
  .rename(columns=lambda c: c.replace('.', '_'))
  .assign(age=lambda df_:df_.age.str.slice(0,2).astype(float)
             .astype('Int64'),
         are_you_datascientist=lambda df_:df_.are_you_datascientist
             .replace({'Yes': True, 'No': False, np.nan: False}),
         company_size=lambda df_:df_.company_size.replace({
             'Just me': 1, 'Not sure': np.nan, 
             'More than 5,000': 5000, '2–10': 2, '11–50':11,
             '51–500': 51, '501–1,000':501,
             '1,001–5,000':1001}).astype('Int64'),
         country_live=lambda df_:df_.country_live.astype('category'),
         employment_status=lambda df_:df_.employment_status
              .fillna('Other').astype('category'),
         is_python_main=lambda df_:df_.is_python_main
              .astype('category'),
         team_size=lambda df_:df_.team_size
             .str.split(r'-', n=1, expand=True)
             .iloc[:,0].replace('More than 40 people', 41)
             .where(df_.company_size!=1, 1).astype(float),
         years_of_coding=lambda df_:df_.years_of_coding
             .replace('Less than 1 year', .5).str.extract(r'(\d+)')
             .astype(float),
         python_years=lambda df_:df_.python_years
             .replace('Less than 1 year', .5).str.extract(r'(\d+)')
             .astype(float),
         python3_ver=lambda df_:df_.python3_version_most
              .str.replace('_', '.').str.extract(r'(\d\.\d)')
              .astype(float),
         use_python_most=lambda df_:df_.use_python_most
              .fillna('Unknown')
        )
  .assign(team_size=lambda df_:predict_col(df_, 'team_size')
          .astype(int))
  .drop(columns=['python2_version_most'])
  .dropna()
)

Learning rate set to 0.5
0:	learn: 2.9695218	total: 105ms	remaining: 2s
1:	learn: 2.8766539	total: 144ms	remaining: 1.3s
2:	learn: 2.8387189	total: 188ms	remaining: 1.06s
3:	learn: 2.8028751	total: 220ms	remaining: 878ms
4:	learn: 2.7899957	total: 250ms	remaining: 749ms
5:	learn: 2.7749439	total: 280ms	remaining: 654ms
6:	learn: 2.7719128	total: 309ms	remaining: 574ms
7:	learn: 2.7649792	total: 339ms	remaining: 509ms
8:	learn: 2.7649588	total: 363ms	remaining: 444ms
9:	learn: 2.7630617	total: 399ms	remaining: 399ms
10:	learn: 2.7625779	total: 428ms	remaining: 350ms
11:	learn: 2.7515902	total: 457ms	remaining: 305ms
12:	learn: 2.7513459	total: 485ms	remaining: 261ms
13:	learn: 2.7445634	total: 514ms	remaining: 220ms
14:	learn: 2.7443257	total: 542ms	remaining: 181ms
15:	learn: 2.7423142	total: 571ms	remaining: 143ms
16:	learn: 2.7419144	total: 599ms	remaining: 106ms
17:	learn: 2.7399388	total: 628ms	remaining: 69.8ms
18:	learn: 2.7384297	total: 655ms	remaining: 34.5ms
19:	learn: 2.73835

In [28]:
jb2

,age,are_you_datascientist,company_size,country_live,employment_status,first_learn_about_main_ide,how_often_use_main_ide,ide_main,is_python_main,job_team,main_purposes,missing_features_main_ide,nps_main_ide,python_years,python3_version_most,several_projects,team_size,use_python_most,years_of_coding,python3_ver
1,21,True,5000,India,Fully employed by a company / organization,School / University,Daily,VS Code,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",8.0,3.0,Python 3_6,"Yes, I work on one main and several side projects",2,Software prototyping,3.0,3.6
2,30,False,5000,United States,Fully employed by a company / organization,Friend / Colleague,Daily,Vim,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",10.0,3.0,Python 3_6,"Yes, I work on one main and several side projects",4,DevOps / System administration / Writing autom...,3.0,3.6
10,21,False,51,Other country,Fully employed by a company / organization,School / University,Daily,IntelliJ IDEA,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",10.0,1.0,Python 3_8,"Yes, I work on one main and several side projects",2,Web development,1.0,3.8
11,21,True,51,United States,Fully employed by a company / organization,Online learning platform / Online course,Daily,PyCharm Community Edition,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",9.0,3.0,Python 3_9,"Yes, I work on many different projects",2,Data analysis,3.0,3.9
13,30,True,5000,Belgium,Fully employed by a company / organization,Social network,Daily,VS Code,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",10.0,6.0,Python 3_7,"Yes, I work on many different projects",2,Data analysis,3.0,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54456,30,False,1001,Turkey,Fully employed by a company / organization,Friend / Colleague,Daily,PyCharm Community Edition,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",9.0,1.0,Python 3_6,"Yes, I work on many different projects",5,Machine learning,6.0,3.6
54457,21,False,2,Russian Federation,Fully employed by a company / organization,School / University,Daily,Vim,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",10.0,6.0,Python 3_6,"Yes, I work on many different projects",2,Data analysis,1.0,3.6
54459,21,False,1,Russian Federation,Self-employed (a person earning income directl...,Friend / Colleague,Daily,PyCharm Professional Edition,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",10.0,3.0,Python 3_7,"Yes, I work on many different projects",1,Web development,6.0,3.7
54460,30,True,51,Spain,Fully employed by a company / organization,Search engines,Daily,Other,Yes,Work on your own project(s) independently,Both for work and personal,Yes – Please list:,3.0,6.0,Python 3_7,"Yes, I work on many different projects",4,Data analysis,3.0,3.7


In [52]:
def get_uniq_cols(jb):
    counter = collections.defaultdict(list)
    for col in sorted(jb.columns):
        period_count = col.count('.')
        if period_count >= 2:
            part_end = 2
        else:
            part_end = 1
        parts = col.split('.')[:part_end]
        counter['.'.join(parts)].append(col)
    uniq_cols = []
    for cols in counter.values():
        if len(cols) == 1:
            uniq_cols.extend(cols)
    return uniq_cols

In [53]:
def prep_for_ml(df):
    # remove pandas types
    return (df
           .assign(**{col:df[col].astype(float)
                     for col in df.select_dtypes('number')},
                  **{col:df[col].astype(str).fillna('')
                    for col in df.select_dtypes(['object', 'category'])})
           )

In [54]:
def predict_col(df, col):
    df = prep_for_ml(df)
    missing = df.query(f'~{col}.isna()')
    cat_idx = []
    for i, typ in enumerate(df.drop(columns=[col]).dtypes):
        if str(typ) == 'object':
              cat_idx.append(i)
    X = (missing
        .drop(columns=[col])
        .values
        )
    y = missing[col]
    model = cb.CatBoostRegressor(iterations=20, cat_features=cat_idx)
    model.fit(X, y, cat_features=cat_idx)
    pred = model.predict(df.drop(columns=[col]))
    return df[col].where(~df[col].isna(), pred)

In [55]:
def tweak_jb(jb):
     uniq_cols = get_uniq_cols(jb)
     return (jb
         [uniq_cols]
         .rename(columns=lambda c: c.replace('.', '_'))
         .assign(age=lambda df_:df_.age.str.slice(0,2).astype(float)
                    .astype('Int64'),
                are_you_datascientist=lambda df_:df_
                    .are_you_datascientist
                    .replace({'Yes': True, 'No': False, np.nan: False}),
                company_size=lambda df_:df_.company_size.replace({
                    'Just me': 1, 'Not sure': np.nan,
                    'More than 5,000': 5000, '2–10': 2, '11–50':11,
                    '51–500': 51, '501–1,000':501,
                    '1,001–5,000':1001}).astype('Int64'),
                country_live=lambda df_:df_.country_live
                    .astype('category'),
                employment_status=lambda df_:df_.employment_status
                     .fillna('Other').astype('category'),
                is_python_main=lambda df_:df_.is_python_main
                     .astype('category'),
                team_size=lambda df_:df_.team_size
                    .str.split(r'-', n=1, expand=True)
                    .iloc[:,0].replace('More than 40 people', 41)
                    .where(df_.company_size!=1, 1).astype(float),
                years_of_coding=lambda df_:df_.years_of_coding
                    .replace('Less than 1 year', .5)
                    .str.extract(r'(\d+)').astype(float),
                python_years=lambda df_:df_.python_years
                    .replace('Less than 1 year', .5)
                    .str.extract(r'(\d+)').astype(float),
                python3_ver=lambda df_:df_.python3_version_most
                     .str.replace('_', '.').str.extract(r'(\d\.\d)')
                     .astype(float),
                use_python_most=lambda df_:df_.use_python_most
                     .fillna('Unknown')
               )
        .assign(team_size=lambda df_:predict_col(df_, 'team_size')
             .astype(int))
        .drop(columns=['python2_version_most'])
        .dropna()
    )

In [56]:
url = 'https://github.com/mattharrison/datasets/raw/master/data//2020-jetbrains-python-survey.csv'

In [57]:
jb = pd.read_csv(url)

/tmp/ipykernel_47/1530167600.py:1: DtypeWarning: Columns (152) have mixed types. Specify dtype option on import or set low_memory=False.
  jb = pd.read_csv(url)


In [58]:
jb2 = tweak_jb(jb)

Learning rate set to 0.5
0:	learn: 2.9695218	total: 35ms	remaining: 666ms
1:	learn: 2.8766539	total: 69.6ms	remaining: 626ms
2:	learn: 2.8387189	total: 110ms	remaining: 622ms
3:	learn: 2.8028751	total: 140ms	remaining: 559ms
4:	learn: 2.7899957	total: 169ms	remaining: 507ms
5:	learn: 2.7749439	total: 201ms	remaining: 469ms
6:	learn: 2.7719128	total: 230ms	remaining: 427ms
7:	learn: 2.7649792	total: 266ms	remaining: 400ms
8:	learn: 2.7649588	total: 289ms	remaining: 354ms
9:	learn: 2.7630617	total: 318ms	remaining: 318ms
10:	learn: 2.7625779	total: 347ms	remaining: 284ms
11:	learn: 2.7515902	total: 376ms	remaining: 251ms
12:	learn: 2.7513459	total: 406ms	remaining: 219ms
13:	learn: 2.7445634	total: 435ms	remaining: 186ms
14:	learn: 2.7443257	total: 462ms	remaining: 154ms
15:	learn: 2.7423142	total: 489ms	remaining: 122ms
16:	learn: 2.7419144	total: 517ms	remaining: 91.2ms
17:	learn: 2.7399388	total: 546ms	remaining: 60.6ms
18:	learn: 2.7384297	total: 574ms	remaining: 30.2ms
19:	learn: 2.